# Phase 2A: Interactive Visualizations for Multi-Asset OHLCV Data

Interactive visualizations for all assets exported from Phase 1B across multiple asset types:
- **Traditional Commodities** (Gold, Silver, Oil, Natural Gas): DeFi perpetuals vs TradFi futures
- **Traditional Equities** (AAPL, GOOGL, MSFT, NVDA, TSLA): DeFi perpetuals vs TradFi stocks
- **Crypto Coins** (BTC, ETH, SOL, etc.): DEX perpetuals vs CEX spot markets

**Terminology:**
- For traditional assets: **DeFi** (decentralized perpetuals) vs **TradFi** (traditional markets)
- For crypto assets: **DEX** (decentralized perpetuals) vs **CEX** (centralized exchange spot)

**Visualizations (Organized by Asset Type):**
For each asset type, the notebook presents:
1. **Overview Dashboard** - Summary statistics including volume metrics, price correlation, and data overlap
2. **Volume Comparison** - Notional volume charts showing DeFi/DEX vs TradFi/CEX trading activity

In [1]:
import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

PHASE_1B_DIR = os.path.join("output", "Phase 1B")

# Helper function to get appropriate labels based on asset type
def get_labels(asset_type):
    """Return appropriate labels for onchain/offchain data based on asset type.
    
    - Crypto assets: 'DEX' vs 'CEX' (Decentralized vs Centralized Exchanges)
    - Traditional assets: 'DeFi' vs 'TradFi' (Decentralized vs Traditional Finance)
    """
    if "Crypto" in asset_type:
        return {"onchain": "DEX", "offchain": "CEX", "onchain_long": "DEX", "offchain_long": "Centralized Exchange"}
    else:
        return {"onchain": "DeFi", "offchain": "TradFi", "onchain_long": "DeFi", "offchain_long": "Traditional Finance"}

## Load All Assets from Phase 1B

In [2]:
def load_all_assets():
    """Load all asset Excel files from Phase 1B output directory."""
    assets = {}
    
    # Find all subdirectories (asset types) in Phase 1B output
    asset_type_dirs = [d for d in Path(PHASE_1B_DIR).iterdir() if d.is_dir()]
    
    for asset_type_dir in asset_type_dirs:
        asset_type = asset_type_dir.name
        
        # Find all Excel files in this asset type directory
        excel_files = list(asset_type_dir.glob("*.xlsx"))
        
        for excel_file in excel_files:
            asset_name = excel_file.stem  # Filename without extension
            
            try:
                df = pd.read_excel(excel_file)
                df["time"] = pd.to_datetime(df["time"])
                
                # Count overlapping data points
                overlap_price = df.dropna(subset=["defi_close", "tradfi_close"]).shape[0]
                overlap_volume = df.dropna(subset=["defi_notional_volume", "tradfi_notional_volume"]).shape[0]
                
                assets[asset_name] = {
                    "data": df,
                    "asset_type": asset_type,
                    "total_rows": len(df),
                    "overlap_price": overlap_price,
                    "overlap_volume": overlap_volume,
                }
                
                print(f"{asset_name:<15} ({asset_type:<30}) {len(df):>4} rows, {overlap_price:>3} price overlap, {overlap_volume:>3} volume overlap")
            
            except Exception as e:
                print(f"Error loading {excel_file}: {e}")
    
    return assets

assets = load_all_assets()
print(f"\nTotal assets loaded: {len(assets)}")

# Group assets by type
assets_by_type = {}
for asset_name, asset_info in assets.items():
    asset_type = asset_info["asset_type"]
    if asset_type not in assets_by_type:
        assets_by_type[asset_type] = []
    assets_by_type[asset_type].append(asset_name)

Oil             (Traditional Commodity         )   37 rows,  25 price overlap,  25 volume overlap
Silver          (Traditional Commodity         )   57 rows,  37 price overlap,  37 volume overlap
Natural Gas     (Traditional Commodity         )   22 rows,  15 price overlap,  15 volume overlap
Gold            (Traditional Commodity         )   62 rows,  40 price overlap,  40 volume overlap
BTC             (Crypto Coin                   ) 2003 rows, 2002 price overlap, 2002 volume overlap
SOL             (Crypto Coin                   ) 1977 rows, 1976 price overlap, 1976 volume overlap
LINK            (Crypto Coin                   ) 1990 rows, 1989 price overlap, 1989 volume overlap
ETH             (Crypto Coin                   ) 2003 rows, 2002 price overlap, 2002 volume overlap
ADA             (Crypto Coin                   ) 1990 rows, 1842 price overlap, 1842 volume overlap
META            (Traditional Equity            )   84 rows,  55 price overlap,  55 volume overlap
NVDA      

# Visualizations by Asset Type

For each asset type, we'll create:
1. **Overview Dashboard** - Summary statistics for all assets in the type
2. **Volume Comparison** - Notional volume charts for each asset

In [3]:
def create_overview_dashboard(asset_type, asset_names):
    """Create overview dashboard for a specific asset type."""
    labels = get_labels(asset_type)
    summary_data = []
    
    for asset_name in asset_names:
        asset_info = assets[asset_name]
        df = asset_info["data"]
        
        # Calculate statistics for overlapping period
        overlap_mask = df[["defi_close", "tradfi_close"]].notna().all(axis=1)
        
        if overlap_mask.any():
            overlap_df = df[overlap_mask]
            
            # Average daily notional volume
            onchain_avg_vol = overlap_df["defi_notional_volume"].mean()
            offchain_avg_vol = overlap_df["tradfi_notional_volume"].mean()
            
            # Price correlation
            price_corr = overlap_df["defi_close"].corr(overlap_df["tradfi_close"])
            
            # Average price difference (percentage)
            price_diff_pct = ((overlap_df["defi_close"] - overlap_df["tradfi_close"]) / overlap_df["tradfi_close"] * 100).mean()
            
            summary_data.append({
                "Asset": asset_name,
                "Total Days": asset_info["total_rows"],
                "Overlap Days": asset_info["overlap_price"],
                f"{labels['onchain']} Avg Vol (USD)": onchain_avg_vol,
                f"{labels['offchain']} Avg Vol (USD)": offchain_avg_vol,
                f"Vol Ratio ({labels['offchain']}/{labels['onchain']})": offchain_avg_vol / onchain_avg_vol if onchain_avg_vol > 0 else np.nan,
                "Price Correlation": price_corr,
                "Avg Price Diff %": price_diff_pct,
            })
    
    summary_df = pd.DataFrame(summary_data).sort_values("Asset")
    
    # Format numeric columns for display
    display_df = summary_df.copy()
    
    # Format volume columns (dynamic names)
    for col in display_df.columns:
        if "Avg Vol (USD)" in col:
            display_df[col] = display_df[col].apply(lambda x: f"${x:,.0f}" if not pd.isna(x) else "N/A")
        elif "Vol Ratio" in col:
            display_df[col] = display_df[col].apply(lambda x: f"{x:.1f}x" if not pd.isna(x) else "N/A")
    
    display_df["Price Correlation"] = display_df["Price Correlation"].apply(lambda x: f"{x:.3f}" if not pd.isna(x) else "N/A")
    display_df["Avg Price Diff %"] = display_df["Avg Price Diff %"].apply(lambda x: f"{x:+.2f}%" if not pd.isna(x) else "N/A")
    
    return display_df


def plot_volume_comparison(asset_name, asset_type):
    """Plot volume comparison for a given asset."""
    df = assets[asset_name]["data"]
    labels = get_labels(asset_type)
    
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=(
            f"{labels['onchain']} vs {labels['offchain']} Notional Volume", 
            f"Volume Ratio ({labels['offchain']}/{labels['onchain']})"
        ),
        row_heights=[0.7, 0.3]
    )
    
    # Volume comparison
    fig.add_trace(
        go.Bar(x=df["time"], y=df["defi_notional_volume"], 
              name=f"{labels['onchain']} Volume", 
              marker_color="steelblue", opacity=0.7),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(x=df["time"], y=df["tradfi_notional_volume"], 
              name=f"{labels['offchain']} Volume", 
              marker_color="coral", opacity=0.7),
        row=1, col=1
    )
    
    # Volume ratio
    vol_ratio = df["tradfi_notional_volume"] / df["defi_notional_volume"]
    fig.add_trace(
        go.Scatter(x=df["time"], y=vol_ratio, name="Volume Ratio", 
                  mode="lines+markers", line=dict(color="purple", width=2),
                  showlegend=False),
        row=2, col=1
    )
    fig.add_hline(y=1, line_dash="dash", line_color="gray", opacity=0.5, 
                 annotation_text="1:1 ratio", row=2, col=1)
    
    fig.update_layout(
        title=f"{asset_name} - Volume Comparison",
        height=700,
        hovermode="x unified",
        template="plotly_white",
        barmode="group",
    )
    fig.update_yaxes(title_text="Volume (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Ratio", type="log", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    
    fig.show()


def plot_crypto_volume_comparison(asset_name):
    """Plot volume comparison for crypto assets with dual y-axes (from July 2024, line graphs)."""
    df = assets[asset_name]["data"].copy()
    
    # Filter data from July 2024 onwards
    df = df[df["time"] >= "2024-07-01"]
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add Perpetuals volume line (primary y-axis)
    fig.add_trace(
        go.Scatter(
            x=df["time"], 
            y=df["defi_notional_volume"], 
            name="Perpetuals", 
            mode="lines",
            line=dict(color="steelblue", width=2),
            fill='tozeroy',
            fillcolor='rgba(70, 130, 180, 0.2)'
        ),
        secondary_y=False
    )
    
    # Add Spot Trading volume line (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            x=df["time"], 
            y=df["tradfi_notional_volume"], 
            name="Spot Trading", 
            mode="lines",
            line=dict(color="coral", width=2),
            fill='tozeroy',
            fillcolor='rgba(255, 127, 80, 0.2)'
        ),
        secondary_y=True
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="Perpetuals Volume (USD)", secondary_y=False, title_font=dict(color="steelblue"))
    fig.update_yaxes(title_text="Spot Trading Volume (USD)", secondary_y=True, title_font=dict(color="coral"))
    
    fig.update_layout(
        title=f"{asset_name} - Trading Volume Comparison (July 2024 onwards)",
        height=500,
        hovermode="x unified",
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    fig.show()


def plot_commodity_volume_comparison(asset_name):
    """Plot volume comparison for commodity assets with dual y-axes (line graphs, no ratio)."""
    df = assets[asset_name]["data"].copy()
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add DeFi Perpetuals volume line (primary y-axis)
    fig.add_trace(
        go.Scatter(
            x=df["time"], 
            y=df["defi_notional_volume"], 
            name="DeFi Perpetuals", 
            mode="lines",
            line=dict(color="steelblue", width=2),
            fill='tozeroy',
            fillcolor='rgba(70, 130, 180, 0.2)'
        ),
        secondary_y=False
    )
    
    # Add Traditional Futures volume line (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            x=df["time"], 
            y=df["tradfi_notional_volume"], 
            name="Traditional Futures", 
            mode="lines",
            line=dict(color="coral", width=2),
            fill='tozeroy',
            fillcolor='rgba(255, 127, 80, 0.2)'
        ),
        secondary_y=True
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date")
    fig.update_yaxes(title_text="DeFi Perpetuals Volume (USD)", secondary_y=False, title_font=dict(color="steelblue"))
    fig.update_yaxes(title_text="Traditional Futures Volume (USD)", secondary_y=True, title_font=dict(color="coral"))
    
    fig.update_layout(
        title=f"{asset_name} - Trading Volume Comparison",
        height=500,
        hovermode="x unified",
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    fig.show()

In [4]:
# Get sorted asset types for reference
sorted_asset_types = sorted(assets_by_type.keys())
print("Available asset types:", sorted_asset_types)

Available asset types: ['Crypto Coin', 'Traditional Commodity', 'Traditional Equity']


---
# Crypto Coin Assets
## DEX Perpetuals vs CEX Spot Markets
---

In [5]:
asset_type = "Crypto Coin"
asset_names = sorted(assets_by_type[asset_type])

# Filter data to July 2024 onwards for overview
filtered_summary_data = []

for asset_name in asset_names:
    asset_info = assets[asset_name]
    df = asset_info["data"].copy()
    
    # Filter to July 2024 onwards
    df = df[df["time"] >= "2024-07-01"]
    
    # Calculate statistics for overlapping period
    overlap_mask = df[["defi_close", "tradfi_close"]].notna().all(axis=1)
    
    if overlap_mask.any():
        overlap_df = df[overlap_mask]
        
        # Average daily notional volume
        perp_avg_vol = overlap_df["defi_notional_volume"].mean()
        spot_avg_vol = overlap_df["tradfi_notional_volume"].mean()
        
        # Price correlation
        price_corr = overlap_df["defi_close"].corr(overlap_df["tradfi_close"])
        
        # Average price difference (percentage)
        price_diff_pct = ((overlap_df["defi_close"] - overlap_df["tradfi_close"]) / overlap_df["tradfi_close"] * 100).mean()
        
        filtered_summary_data.append({
            "Asset": asset_name,
            "Days (Jul 2024+)": len(df),
            "Overlap Days": overlap_mask.sum(),
            "Perpetuals Avg Vol (USD)": perp_avg_vol,
            "Spot Trading Avg Vol (USD)": spot_avg_vol,
            "Vol Ratio (Spot/Perp)": spot_avg_vol / perp_avg_vol if perp_avg_vol > 0 else np.nan,
            "Price Correlation": price_corr,
            "Avg Price Diff %": price_diff_pct,
        })

overview_df = pd.DataFrame(filtered_summary_data).sort_values("Asset")

# Format numeric columns for display
display_df = overview_df.copy()
display_df["Perpetuals Avg Vol (USD)"] = display_df["Perpetuals Avg Vol (USD)"].apply(lambda x: f"${x:,.0f}")
display_df["Spot Trading Avg Vol (USD)"] = display_df["Spot Trading Avg Vol (USD)"].apply(lambda x: f"${x:,.0f}")
display_df["Vol Ratio (Spot/Perp)"] = display_df["Vol Ratio (Spot/Perp)"].apply(lambda x: f"{x:.1f}x" if not pd.isna(x) else "N/A")
display_df["Price Correlation"] = display_df["Price Correlation"].apply(lambda x: f"{x:.3f}" if not pd.isna(x) else "N/A")
display_df["Avg Price Diff %"] = display_df["Avg Price Diff %"].apply(lambda x: f"{x:+.2f}%" if not pd.isna(x) else "N/A")

display(display_df)

,Asset,Days (Jul 2024+),Overlap Days,Perpetuals Avg Vol (USD),Spot Trading Avg Vol (USD),Vol Ratio (Spot/Perp),Price Correlation,Avg Price Diff %
0,ADA,591,590,"$11,934,059","$1,063,454,831",89.1x,1.000,-0.04%
1,BTC,591,590,"$2,438,866,497","$49,644,866,577",20.4x,1.000,+0.02%
2,ETH,591,590,"$1,701,249,866","$26,540,955,183",15.6x,1.000,-0.00%
3,LINK,591,590,"$22,743,708","$683,967,615",30.1x,1.000,-0.00%
4,SOL,591,590,"$549,534,645","$4,941,119,098",9.0x,1.000,-0.00%


In [6]:
asset_type = "Crypto Coin"
asset_names = sorted(assets_by_type[asset_type])

for asset_name in asset_names:
    plot_crypto_volume_comparison(asset_name)

---
# Traditional Commodity Assets
## DeFi Perpetuals vs TradFi Futures
---

### Overview Dashboard - Traditional Commodities

In [7]:
asset_type = "Traditional Commodity"
asset_names = sorted(assets_by_type[asset_type])

overview_df = create_overview_dashboard(asset_type, asset_names)
display(overview_df)

,Asset,Total Days,Overlap Days,DeFi Avg Vol (USD),TradFi Avg Vol (USD),Vol Ratio (TradFi/DeFi),Price Correlation,Avg Price Diff %
0,Gold,62,40,"$306,999,007","$24,777,687",0.1x,0.982,+0.46%
1,Natural Gas,22,15,"$22,945,528","$982,013",0.0x,0.524,-15.19%
2,Oil,37,25,"$16,894,083","$21,491,556",1.3x,0.925,+3.80%
3,Silver,57,37,"$1,629,820,870","$52,096",0.0x,0.973,+1.47%


In [8]:
asset_type = "Traditional Commodity"
asset_names = sorted(assets_by_type[asset_type])

# Create custom overview with descriptive labels
summary_data = []

for asset_name in asset_names:
    asset_info = assets[asset_name]
    df = asset_info["data"]
    
    # Calculate statistics for overlapping period
    overlap_mask = df[["defi_close", "tradfi_close"]].notna().all(axis=1)
    
    if overlap_mask.any():
        overlap_df = df[overlap_mask]
        
        # Average daily notional volume
        defi_avg_vol = overlap_df["defi_notional_volume"].mean()
        tradfi_avg_vol = overlap_df["tradfi_notional_volume"].mean()
        
        # Price correlation
        price_corr = overlap_df["defi_close"].corr(overlap_df["tradfi_close"])
        
        # Average price difference (percentage)
        price_diff_pct = ((overlap_df["defi_close"] - overlap_df["tradfi_close"]) / overlap_df["tradfi_close"] * 100).mean()
        
        summary_data.append({
            "Asset": asset_name,
            "Total Days": asset_info["total_rows"],
            "Overlap Days": asset_info["overlap_price"],
            "DeFi Perpetuals Avg Vol (USD)": defi_avg_vol,
            "Traditional Futures Avg Vol (USD)": tradfi_avg_vol,
            "Vol Ratio (TradFi/DeFi)": tradfi_avg_vol / defi_avg_vol if defi_avg_vol > 0 else np.nan,
            "Price Correlation": price_corr,
            "Avg Price Diff %": price_diff_pct,
        })

overview_df = pd.DataFrame(summary_data).sort_values("Asset")

# Format numeric columns for display
display_df = overview_df.copy()
display_df["DeFi Perpetuals Avg Vol (USD)"] = display_df["DeFi Perpetuals Avg Vol (USD)"].apply(lambda x: f"${x:,.0f}")
display_df["Traditional Futures Avg Vol (USD)"] = display_df["Traditional Futures Avg Vol (USD)"].apply(lambda x: f"${x:,.0f}")
display_df["Vol Ratio (TradFi/DeFi)"] = display_df["Vol Ratio (TradFi/DeFi)"].apply(lambda x: f"{x:.1f}x" if not pd.isna(x) else "N/A")
display_df["Price Correlation"] = display_df["Price Correlation"].apply(lambda x: f"{x:.3f}" if not pd.isna(x) else "N/A")
display_df["Avg Price Diff %"] = display_df["Avg Price Diff %"].apply(lambda x: f"{x:+.2f}%" if not pd.isna(x) else "N/A")

display(display_df)

,Asset,Total Days,Overlap Days,DeFi Perpetuals Avg Vol (USD),Traditional Futures Avg Vol (USD),Vol Ratio (TradFi/DeFi),Price Correlation,Avg Price Diff %
0,Gold,62,40,"$306,999,007","$24,777,687",0.1x,0.982,+0.46%
1,Natural Gas,22,15,"$22,945,528","$982,013",0.0x,0.524,-15.19%
2,Oil,37,25,"$16,894,083","$21,491,556",1.3x,0.925,+3.80%
3,Silver,57,37,"$1,629,820,870","$52,096",0.0x,0.973,+1.47%


In [9]:
asset_type = "Traditional Commodity"
asset_names = sorted(assets_by_type[asset_type])

for asset_name in asset_names:
    plot_commodity_volume_comparison(asset_name)

### Overview Dashboard - Traditional Equities

In [10]:
asset_type = "Traditional Equity"
asset_names = sorted(assets_by_type[asset_type])

overview_df = create_overview_dashboard(asset_type, asset_names)
display(overview_df)

,Asset,Total Days,Overlap Days,DeFi Avg Vol (USD),TradFi Avg Vol (USD),Vol Ratio (TradFi/DeFi),Price Correlation,Avg Price Diff %
0,AAPL,84,54,"$3,369,643","$12,910,090,224",3831.3x,0.998,+0.06%
1,AMZN,84,55,"$7,779,450","$10,357,381,164",1331.4x,0.943,-0.15%
2,COIN,82,52,"$8,269,214","$2,140,279,785",258.8x,0.999,+0.09%
3,GOOGL,84,55,"$16,360,099","$11,931,137,694",729.3x,0.986,+0.11%
4,META,84,55,"$5,882,486","$11,016,479,810",1872.8x,0.958,+0.08%
5,MSFT,84,55,"$4,134,775","$14,128,871,635",3417.1x,0.985,-0.09%
6,NVDA,92,61,"$26,647,553","$33,153,008,442",1244.1x,0.955,+0.05%
7,TSLA,91,60,"$24,848,330","$31,503,478,552",1267.8x,0.994,+0.12%


### Volume Comparisons - Traditional Equities

In [11]:
asset_type = "Traditional Equity"
asset_names = sorted(assets_by_type[asset_type])

for asset_name in asset_names:
    plot_volume_comparison(asset_name, asset_type)